Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Text2SQL with AutoGen & Azure OpenAI

This notebook demonstrates how the AutoGen Agents can be integrated with Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

### Dependencies

To install dependencies for this demo:

`uv sync --package autogen_text_2_sql`

`uv add --editable text_2_sql_core`

In [ ]:
# This is only needed for this notebook to work
import sys
from pathlib import Path

# Add the parent directory of `src` to the path
sys.path.append(str(Path.cwd() / "src"))

In [ ]:
import dotenv
import logging
from autogen_agentchat.ui import Console
from autogen_text_2_sql.autogen_text_2_sql import AutoGenText2Sql

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
dotenv.load_dotenv()

## Bot setup

In [ ]:
agentic_text_2_sql = AutoGenText2Sql(engine_specific_rules="Use TOP X to limit the number of rows returned instead of LIMIT X. NEVER USE LIMIT X as it produces a syntax error.", use_case="Analysing sales data across product categories.").agentic_flow

In [ ]:
result = agentic_text_2_sql.run_stream(task="What are the total number of sales within 2008 for the mountain bike product line?")

In [ ]:
await Console(result)